In [1]:
import pandas as pd
import numpy as np
import datetime
import os

datetime.datetime.now()

datetime.datetime(2020, 2, 4, 18, 0, 32, 289419)

In [2]:
df_store_type=pd.ExcelFile("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/other_input/BL Store List Report from Dom 12.20.19.xlsx")
df_store_type=df_store_type.parse("Store List",dtype=str)
df_store_type=df_store_type[["Store",'SOTF']]
df_store_type=df_store_type[df_store_type['SOTF']=="Yes"]
df_store_type=df_store_type[df_store_type['Store']!="TBD"]
print(len(df_store_type),df_store_type['Store'].nunique())
df_store_type=df_store_type.rename(columns={"Store":"location_id","SOTF":"Store_Type"})
df_store_type.head(2)

479 479


,location_id,Store_Type
0,1,Yes
8,49,Yes


In [3]:
file_furinture_sales_label=pd.ExcelFile("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/BL_output_store_type_furniture_3_months_JL_2020-01-16.xlsx")
df_store_furniture_label=file_furinture_sales_label.parse('store_type_recent_3_months',dtype=str)

df_store_furniture_label=df_store_furniture_label[['location_id','store_furniture_group','rank','furniture_sales','total_fur_share']]

print(df_store_furniture_label.shape,df_store_furniture_label['location_id'].nunique())

# df_store_furniture_label=df_store_furniture_label.rename(columns={"location_id":"allocated_store"})
df_store_furniture_label['store_furniture_group']=df_store_furniture_label['store_furniture_group'].apply(lambda x: "Furiniture"+x)
df_store_furniture_label.head(2)

(1426, 5) 1426


,location_id,store_furniture_group,rank,furniture_sales,total_fur_share
0,1392,FurinitureHigh,1,639520.8300000002,0.466022037205638
1,5388,FurinitureHigh,2,670603.4500000003,0.4449548556274898


In [4]:
store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20200201-134613-934.txt",dtype=str,sep="|")
store_list=store_list[['location_id','address_line_1','address_line_2','city_nm','state_nm','zip_cd','latitude_meas','longitude_meas']]
store_list['latitude_meas']=store_list['latitude_meas'].astype(float)
store_list['longitude_meas']=store_list['longitude_meas'].astype(float)
store_list=store_list[store_list['location_id']!="145"]
store_list=store_list[store_list['location_id']!="6990"]


store_list=pd.merge(store_list,df_store_type,on="location_id",how="left")
store_list=pd.merge(store_list,df_store_furniture_label,on="location_id",how="left")


store_list.head(2)


,location_id,address_line_1,address_line_2,city_nm,state_nm,zip_cd,latitude_meas,longitude_meas,Store_Type,store_furniture_group,rank,furniture_sales,total_fur_share
0,3,2708 PEACH ORCHARD RD,NaN,AUGUSTA,GA,30906-2408,33.421570,-82.018921,NaN,FurinitureHigh,225,301461.2,0.3134393962307467
1,30,818 S MAIN ST,NaN,BOWLING GREEN,OH,43402-4601,41.363132,-83.650909,NaN,FurinitureHigh,196,247426.4699999999,0.3178732055928086


In [5]:
store_list.shape,store_list['location_id'].nunique()

((1404, 13), 1404)

In [6]:
store_list['rank']=store_list['rank'].astype(int)
store_list['Store_Type']=np.where(store_list['Store_Type']=="Yes","SOTF","Legacy")

df_output=store_list[store_list['rank']<=702]
df_output=df_output[df_output['Store_Type']=="SOTF"]
df_output.shape

(268, 13)

In [8]:
writer=pd.ExcelWriter("./BL_store_addresses_SOTF_top50_Furniture_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")

df_output.to_excel(writer,"addresses_needed",index=False)
store_list.to_excel(writer,"all_stores",index=False)

writer.save()
